In [ ]:
require 'torch'

In [ ]:
require 'nn'
require 'optim'

In [ ]:
require 'moses'

In [ ]:
torch.manualSeed(1234)

In [ ]:
n = 5

In [ ]:
X = torch.rand(n, n)

In [ ]:
X

In [ ]:
X = X*X:t()

In [ ]:
X

In [ ]:
X:add(0.001, torch.eye(n))

In [ ]:
X

In [ ]:
b = torch.rand(n)

In [ ]:
function J(y, z)
    return 0.5*z:dot(y*z) - b:dot(z)
end

In [ ]:
print( J(X, torch.rand(n)))

In [ ]:
print( J(torch.rand(n, n), torch.rand(n)))

In [ ]:
xs = torch.inverse(X)*b

In [ ]:
print(string.format('J(x^*) = %g', J(X, xs)))

In [ ]:
function dJ(A, z, b)
  return A*z-b
end

In [ ]:
x = torch.rand(n)

lr = 0.01
for i=1,20000 do
  x = x - dJ(X, x, b)*lr
  print(string.format('at iter %d J(x) = %f', i, J(X, x)))
end

In [ ]:
do
   local A = torch.rand(N, N)
   print(A)
end
print(A)

In [ ]:
do
   local neval = 0
   function JdJ(x)
      local Jx = J(x)
      neval = neval + 1
      print(string.format('after %d evaluations J(x) = %f', neval, Jx))
      return Jx, dJ(x)
   end
end

In [ ]:
xs = torch.rand(100, 5)
e = torch.rand(100, 1)
b = torch.randn(5)
yhat = xs*b
y = yhat + e

In [ ]:
linLayer = nn.Linear(5, 10)
linLayer2 = nn.Linear(10, 50)
linLayer3 = nn.Linear(50, 1)

In [ ]:
model = nn.Sequential()
model:add(linLayer)
model:add(linLayer2)
model:add(linLayer3)

In [ ]:
criterion = nn.MSECriterion()

In [ ]:
x, dl_dx = model:getParameters()

In [ ]:
data = torch.Tensor(xs, y)

In [ ]:
logger = optim.Logger('loss_log.txt')


In [ ]:
feval = function(x_new)
    if x ~= x_new then
        x:copy(x_new)
    end

    _nidx_ = (_nidx_ or 0) + 1
    if _nidx_ > (#data)[1] then _nidx_ = 1 end

    local sample = data[_nidx_]
    local target = sample[{ 6 }]        -- this funny looking syntax allows
    local inputs = sample[{ {1,2,3,4,5} }]    -- slicing of arrays.

    dl_dx:zero()
    local loss_x = criterion:forward(model:forward(inputs), target)
    model:backward(inputs, criterion:backward(model.output, target))
    if false then
        local p = model:getParameters()
        print(string.format(
                "parameters %0.2f %0.2f %0.2f %0.2f %0.2f %0.2f %0.2f %0.2f %0.2f", 
                p[1], p[2], p[3], p[4], p[5], p[6], p[7], p[8], p[9])
        )
    end

    -- return loss(x) and dloss/dx
    return loss_x, dl_dx
end


In [ ]:
sgd_params = {
   learningRate = 1e-3,
   learningRateDecay = 1e-4,
   weightDecay = 0,
   momentum = 0
}


In [ ]:
for i = 1,1e4 do
   for i = 1,(#data)[1] do
      _,fs = optim.sgd(feval, x, sgd_params)
      current_loss = current_loss + fs[1]
   end
   current_loss = current_loss / (#data)[1]
   print('current loss = ' .. current_loss)
   
   logger:add{['training error'] = current_loss}
   logger:style{['training error'] = '-'}
   logger:plot()  
end


In [ ]:
data = torch.Tensor{
   {40,  6,  4, 8},
   {44, 10,  4, 7},
   {46, 12,  5, 13},
   {48, 14,  7, 4},
   {52, 16,  9, 20},
   {58, 18, 12, 8},
   {60, 22, 14, 5},
   {68, 24, 20, 13},
   {74, 26, 21, 20},
   {80, 32, 24, 21}
--    {40,  6,  4},
--    {44, 10,  4},
--    {46, 12,  5},
--    {48, 14,  7},
--    {52, 16,  9},
--    {58, 18, 12},
--    {60, 22, 14},
--    {68, 24, 20},
--    {74, 26, 21},
--    {80, 32, 24}
}
logger = optim.Logger('loss_log.txt')


model = nn.Sequential()                 -- define the container
ninputs = 2; noutputs = 1
model:add(nn.Linear(ninputs, noutputs)) -- define the only module
criterion = nn.MSECriterion()

x, dl_dx = model:getParameters()

feval = function(x_new)
   if x ~= x_new then
      x:copy(x_new)
   end
   _nidx_ = (_nidx_ or 0) + 1
   if _nidx_ > (#data)[1] then _nidx_ = 1 end

   local sample = data[_nidx_]
   local target = sample[{ {1} }]      -- this funny looking syntax allows
   local inputs = sample[{ {2,3,4} }]    -- slicing of arrays.
--    local inputs = sample[{ {2,3} }]    -- slicing of arrays.

   dl_dx:zero()
   local loss_x = criterion:forward(model:forward(inputs), target)
   model:backward(inputs, criterion:backward(model.output, target))
   return loss_x, dl_dx
end

sgd_params = {
   learningRate = 1e-3,
   learningRateDecay = 1e-4,
   weightDecay = 0,
   momentum = 0
}

for i = 1,1e4 do
   current_loss = 0
   for i = 1,(#data)[1] do
      _,fs = optim.sgd(feval,x,sgd_params)
      current_loss = current_loss + fs[1]
   end

   current_loss = current_loss / (#data)[1]
   print('current loss = ' .. current_loss)
   
   logger:add{['training error'] = current_loss}
   logger:style{['training error'] = '-'}
   logger:plot()  
end

text = {40.32, 42.92, 45.33, 48.85, 52.37, 57, 61.82, 69.78, 72.19, 79.42}

print('id  approx   text')
for i = 1,(#data)[1] do
   local myPrediction = model:forward(data[i][{{2,3}}])
   ---print(string.format("%2d  %6.2f %6.2f", i, myPrediction[1], text[i]))
end